In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from fairlearn.metrics import (
    MetricFrame, 
    demographic_parity_difference, 
    equalized_odds_difference, 
    selection_rate
)

# Load and preprocess COMPAS dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\compas-scores-raw.csv")

# Filtering
df = df[
    (df['IsCompleted'] == 1) &
    (df['ScoreText'].isin(['Low', 'Medium', 'High']))
]
df['two_year_recid'] = df['ScoreText'].map({'Low': 0, 'Medium': 1, 'High': 1})
df = df[df['Ethnic_Code_Text'].isin(['African-American', 'Caucasian'])]
df['race_binary'] = df['Ethnic_Code_Text'].map({'Caucasian': 0, 'African-American': 1})
df['sex_binary'] = df['Sex_Code_Text'].map({'Male': 1, 'Female': 0})

df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'], errors='coerce')
df['Screening_Date'] = pd.to_datetime(df['Screening_Date'], errors='coerce')
df['age'] = (df['Screening_Date'] - df['DateOfBirth']).dt.days // 365
df = df.dropna(subset=['age', 'LegalStatus', 'CustodyStatus', 'RecSupervisionLevel'])

# One-hot encode
df = pd.get_dummies(df, columns=['LegalStatus', 'CustodyStatus', 'RecSupervisionLevel'], drop_first=True)

# Features and targets
features = ['sex_binary', 'age'] + [col for col in df.columns if col.startswith(('LegalStatus_', 'CustodyStatus_', 'RecSupervisionLevel_'))]
X = df[features]
y = df['two_year_recid']
protected = df['race_binary']

# Normalize
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split
X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
    X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Class weights
import numpy as np
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Define FFNN
class FFNN(nn.Module):
    def __init__(self, input_dim):
        super(FFNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.network(x)

# Initialize model, loss, optimizer
model = FFNN(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    y_logits = model(X_test_tensor)
    y_pred = torch.argmax(y_logits, axis=1).numpy()

# Performance
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print("Feedforward Neural Network Performance on COMPAS:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness Metrics
fair_metrics = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=prot_test
)

spd = demographic_parity_difference(y_test, y_pred, sensitive_features=prot_test)
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=prot_test)
di_ratio = fair_metrics.by_group['selection_rate'].max() / fair_metrics.by_group['selection_rate'].min()

print("\nFairness Metrics (FFNN):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


C:\Users\Arhamsoft\AppData\Local\Temp\ipykernel_8972\3099806426.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'], errors='coerce')
C:\Users\Arhamsoft\AppData\Local\Temp\ipykernel_8972\3099806426.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Screening_Date'] = pd.to_datetime(df['Screening_Date'], errors='coerce')


Feedforward Neural Network Performance on COMPAS:
Accuracy : 0.7992
Precision: 0.6790
Recall   : 0.8003
F1 Score : 0.7346

Fairness Metrics (FFNN):
Statistical Parity Difference : 0.2699
Equal Opportunity Difference : 0.1744
Disparate Impact Ratio       : 2.0345


In [2]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Prepare DataFrame for AIF360
df_ffnn = pd.concat([X_scaled.reset_index(drop=True), y.reset_index(drop=True), protected.reset_index(drop=True)], axis=1)
df_ffnn = df_ffnn.rename(columns={'race_binary': 'race', 'two_year_recid': 'target'})

# Create AIF360 BinaryLabelDataset
bld = BinaryLabelDataset(
    df=df_ffnn,
    label_names=["target"],
    protected_attribute_names=["race"],
    favorable_label=0,
    unfavorable_label=1
)

# Split into train/test
train_bld, test_bld = bld.split([0.7], shuffle=True)

# Apply Reweighing
rw = Reweighing(privileged_groups=[{"race": 0}], unprivileged_groups=[{"race": 1}])
train_rw = rw.fit_transform(train_bld)

# Extract weighted training data
X_train_rw = torch.tensor(train_rw.features, dtype=torch.float32)
y_train_rw = torch.tensor(train_rw.labels.ravel().astype(int), dtype=torch.long)  # ensure labels are 0/1
sample_weights = torch.tensor(train_rw.instance_weights, dtype=torch.float32)

# DataLoader with instance weights
train_dataset_rw = TensorDataset(X_train_rw, y_train_rw, sample_weights)
train_loader_rw = DataLoader(train_dataset_rw, batch_size=128, shuffle=True)

# Redefine model (make sure architecture matches input size)
model_rw = FFNN(input_dim=X_train_rw.shape[1])
optimizer = optim.Adam(model_rw.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduction='none')  # allow per-instance weighting

# Train model with reweighted loss
for epoch in range(100):
    model_rw.train()
    for xb, yb, wb in train_loader_rw:
        optimizer.zero_grad()
        logits = model_rw(xb)
        loss = criterion(logits, yb)
        weighted_loss = (loss * wb).mean()
        weighted_loss.backward()
        optimizer.step()

# Evaluation
model_rw.eval()
X_test_tensor = torch.tensor(test_bld.features, dtype=torch.float32)
y_test_array = test_bld.labels.ravel().astype(int)
prot_test_array = test_bld.protected_attributes.ravel().astype(int)

with torch.no_grad():
    logits = model_rw(X_test_tensor)
    y_pred_rw = torch.argmax(logits, axis=1).numpy()

# Performance Metrics
acc = accuracy_score(y_test_array, y_pred_rw)
prec = precision_score(y_test_array, y_pred_rw, zero_division=0)
rec = recall_score(y_test_array, y_pred_rw, zero_division=0)
f1 = f1_score(y_test_array, y_pred_rw, zero_division=0)

print("Reweighed FFNN Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness Metrics
fair_metrics_rw = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_test_array,
    y_pred=y_pred_rw,
    sensitive_features=prot_test_array
)

spd = demographic_parity_difference(y_test_array, y_pred_rw, sensitive_features=prot_test_array)
eod = equalized_odds_difference(y_test_array, y_pred_rw, sensitive_features=prot_test_array)

# Prevent division by zero in Disparate Impact Ratio
sr_values = fair_metrics_rw.by_group['selection_rate']
di_ratio = sr_values.max() / sr_values.min() if sr_values.min() > 0 else float('nan')

print("\nFairness Metrics (Reweighing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


pip install 'aif360[inFairness]'


Reweighed FFNN Performance:
Accuracy : 0.8028
Precision: 0.7033
Recall   : 0.7530
F1 Score : 0.7273

Fairness Metrics (Reweighing):
Statistical Parity Difference : 0.2120
Equal Opportunity Difference : 0.1132
Disparate Impact Ratio       : 1.8269


In [3]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate
)
import numpy as np
import pandas as pd

# Prepare AIF360 dataset again
df_adv = pd.concat([X_scaled.reset_index(drop=True), y.reset_index(drop=True), protected.reset_index(drop=True)], axis=1)
df_adv = df_adv.rename(columns={'race_binary': 'race', 'two_year_recid': 'target'})

bld_full = BinaryLabelDataset(
    df=df_adv,
    label_names=["target"],
    protected_attribute_names=["race"],
    favorable_label=0,
    unfavorable_label=1
)

train_bld_adv, test_bld_adv = bld_full.split([0.7], shuffle=True)

# Adversarial Debiasing model
sess = tf.Session()
adv_debiasor = AdversarialDebiasing(
    privileged_groups=[{"race": 0}],
    unprivileged_groups=[{"race": 1}],
    scope_name='adv_debiasing_ffnn',
    sess=sess,
    num_epochs=50,
    batch_size=128,
    debias=True
)

adv_debiasor.fit(train_bld_adv)

# Predict on test set
pred_bld_adv = adv_debiasor.predict(test_bld_adv)

# Convert predictions to NumPy
y_true = test_bld_adv.labels.ravel()
y_pred = pred_bld_adv.labels.ravel()
prot_attr = test_bld_adv.protected_attributes.ravel()

# Evaluate Performance
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("Adversarial Debiasing (FFNN) Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness metrics
fair_metrics_adv = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=prot_attr
)

spd = demographic_parity_difference(y_true, y_pred, sensitive_features=prot_attr)
eod = equalized_odds_difference(y_true, y_pred, sensitive_features=prot_attr)
di_ratio = fair_metrics_adv.by_group['selection_rate'].max() / fair_metrics_adv.by_group['selection_rate'].min()

print("\nFairness Metrics (Adversarial Debiasing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.698446; batch adversarial loss: 0.707416
epoch 0; iter: 200; batch classifier loss: 0.494751; batch adversarial loss: 0.690506
epoch 1; iter: 0; batch classifier loss: 0.506592; batch adversarial loss: 0.682501
epoch 1; iter: 200; batch classifier loss: 0.449994; batch adversarial loss: 0.668348
epoch 2; iter: 0; batch classifier loss: 0.499583; batch adversarial loss: 0.677852
epoch 2; iter: 200; batch classifier loss: 0.426547; batch adversarial loss: 0.653829
epoch 3; iter: 0; batch classifier loss: 0.459311; batch adversarial loss: 0.681259
epoch 3; iter: 200; batch classifier loss: 0.466519; batch adversarial loss: 0.684293
epoch 4; iter: 0; batch classifier loss: 0.372865; batch adversarial loss: 0.664264
epoch 4; iter: 200; batch classifier loss: 0.447506; batch adversarial loss: 0.676459
epoch 5; iter: 0; batch classifier loss: 0.500453; batch adversarial loss: 0.681883
epoch 5; iter: 200; batch classifier loss: 0.379989; batch adversa

In [7]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate
)

# Recreate original test dataset from FFNN output
df_eqodds = pd.concat([
    pd.DataFrame(X_test_tensor.numpy(), columns=X_train.columns).reset_index(drop=True),
    pd.Series(y_test.values, name='target').reset_index(drop=True),
    pd.Series(prot_test.values, name='race').reset_index(drop=True)
], axis=1)

test_bld_eq = BinaryLabelDataset(
    df=df_eqodds,
    label_names=["target"],
    protected_attribute_names=["race"],
    favorable_label=0,
    unfavorable_label=1
)

# Create predicted label dataset (FFNN baseline)
pred_bld_eq = test_bld_eq.copy()
pred_bld_eq.labels = y_pred.reshape(-1, 1)

# Apply Equalized Odds Postprocessing
eq_odds = EqOddsPostprocessing(
    privileged_groups=[{"race": 0}],
    unprivileged_groups=[{"race": 1}],
    seed=42
)
eq_odds = eq_odds.fit(test_bld_eq, pred_bld_eq)
pred_eqodds = eq_odds.predict(pred_bld_eq)

# Convert back to NumPy
y_eq = pred_eqodds.labels.ravel()
prot_attr = test_bld_eq.protected_attributes.ravel()
y_true = test_bld_eq.labels.ravel()

# Evaluate Performance
acc = accuracy_score(y_true, y_eq)
prec = precision_score(y_true, y_eq, zero_division=0)
rec = recall_score(y_true, y_eq, zero_division=0)
f1 = f1_score(y_true, y_eq, zero_division=0)

print("Equalized Odds Postprocessing (FFNN) Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness Metrics
fair_metrics_eq = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1": f1_score,
        "selection_rate": selection_rate
    },
    y_true=y_true,
    y_pred=y_eq,
    sensitive_features=prot_attr
)

spd = demographic_parity_difference(y_true, y_eq, sensitive_features=prot_attr)
eod = equalized_odds_difference(y_true, y_eq, sensitive_features=prot_attr)
di_ratio = fair_metrics_eq.by_group['selection_rate'].max() / fair_metrics_eq.by_group['selection_rate'].min()

print("\nFairness Metrics (Equalized Odds Postprocessing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {di_ratio:.4f}")


Equalized Odds Postprocessing (FFNN) Performance:
Accuracy : 0.7433
Precision: 0.6101
Recall   : 0.7227
F1 Score : 0.6617

Fairness Metrics (Equalized Odds Postprocessing):
Statistical Parity Difference : 0.0863
Equal Opportunity Difference : 0.0059
Disparate Impact Ratio       : 1.2372
